In [1]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium

This is a countrywide weather events dataset with more than 5 million events, which covers 49 states of the United States. Examples of weather events are rain, snow, storm, and freezing condition. The data is collected from January 2016 to December 2019, using historical weather reports that exist for airport-based weather stations across the country.

Weather event is a spatiotemporal entity, where such an entity is associated with location and time. Following is the description of available weather event types in this dataset: Cold: The case of having extremely low temperature, with temperature below -23.7 degrees of Celsius.

Fog: The case where there is low visibility condition as a result of fog or haze.

Hail: The case of having solid precipitation including ice pellets and hail.

Rain: The case of having rain, ranging from light to heavy.

Snow: The case of having snow, ranging from light to heavy.

Storm: The extremely windy condition, where the wind speed is at least 60 km/h.

Other Precipitation: Any other type of precipitation which cannot be assigned to previously described event types.

In [14]:
df = pd.read_csv('WeatherEvents_Jan2016-Dec2020.csv')
df.head()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6274206 entries, 0 to 6274205
Data columns (total 13 columns):
EventId           object
Type              object
Severity          object
StartTime(UTC)    object
EndTime(UTC)      object
TimeZone          object
AirportCode       object
LocationLat       float64
LocationLng       float64
City              object
County            object
State             object
ZipCode           float64
dtypes: float64(3), object(10)
memory usage: 622.3+ MB


# data Cleaning

In this project, the duration of each weather event was the key feature used to cluster regions. It was calculated by using event end time minus start time, any single event that lasted more than 30 days was initially eliminated, then any events outside three standard deviation away from mean were also removed.

In [16]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
df['End']=pd.to_datetime(df['EndTime(UTC)'], format=datetimeFormat)
df['Start']=pd.to_datetime(df['StartTime(UTC)'], format=datetimeFormat)

df['Duration']=df['End']-df['Start']
df['Duration'] = df['Duration'].dt.total_seconds()
df['Duration'] = df['Duration']/(60*60) #in hours
df = df[(df['Duration']< 30*24) & (df['Duration'] != 0)] #remove obvious wrong data

df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,End,Start,Duration
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07 00:34:00,2016-01-06 23:14:00,1.333333
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07 04:54:00,2016-01-07 04:14:00,0.666667
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-07 15:34:00,2016-01-07 05:54:00,9.666667
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08 05:54:00,2016-01-08 05:34:00,0.333333
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2016-01-08 15:54:00,2016-01-08 13:54:00,2.000000


In [17]:
df = df[(df['Duration']< 10)]
df['Duration'].size

6206129

#### Data Normalization 
For better comparison of each weather events, all the duration time of them were normalized.
The range 0 to 100, where 
zero means that event never happened in the year,
100 means that event happened all the time during the year.

In [18]:
df2 = df.groupby(['AirportCode','City','State', 'LocationLat', 'LocationLng','Type']).agg({'Duration':['sum']}).reset_index()
df2.columns=pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),("State", " "), ("LocationLat", " "),("LocationLng", " "), ("Type", " "), ("Duration", " ")))

df2.columns = df2.columns.get_level_values(0)
df2['Duration'] = df2['Duration']/(24*4*3.65) #yearly percentage  
df2 = df2.sort_values(by='Duration')

df2.tail()
#dataset for weather durations every city available

,AirportCode,City,State,LocationLat,LocationLng,Type,Duration
8381,KONP,South Beach,OR,44.5804,-124.0579,Rain,16.111967
1776,KBOK,Brookings,OR,42.0744,-124.2900,Rain,17.387177
1104,KAST,Warrenton,OR,46.1569,-123.8825,Rain,18.295044
274,K3TH,Thompson Falls-West End,MT,47.6000,-115.3667,Snow,18.427511
11151,KUIL,Forks,WA,47.9375,-124.5550,Rain,20.579195


In [19]:
df_f = df2.pivot(index='AirportCode', columns='Type', values=['Duration']).reset_index().fillna(0)
df_f.columns=pd.MultiIndex.from_tuples(((' ', 'AirportCode'),(' ', 'Cold'),(' ', 'Fog'),(' ',  'Hail'),(' ', 'Precipitation'),(' ', 'Rain'),(' ', 'Snow'),(' ', 'Storm')))
df_f.columns = df_f.columns.get_level_values(1)

unique = df2[['AirportCode', 'City', 'State', 'LocationLat', 'LocationLng']].sort_values(by='AirportCode').drop_duplicates()
weather = pd.merge(df_f, unique, how='inner', on='AirportCode')

weather.head()
#dataset for weather conditions every city available

,AirportCode,Cold,Fog,Hail,Precipitation,Rain,Snow,Storm,City,State,LocationLat,LocationLng
0,K01M,0.055175,2.501237,0.0,0.240677,8.976598,0.199153,0.000000,Belmont,MS,34.4917,-88.2011
1,K04V,0.090563,0.113680,0.0,0.011653,2.005042,1.913194,0.082572,Saguache,CO,38.0972,-106.1689
2,K04W,0.613489,2.062215,0.0,0.039051,4.820538,3.784484,0.000999,Hinckley,MN,46.0244,-92.8991
3,K06D,1.466895,2.493341,0.0,0.026636,2.018645,5.493721,0.136035,Rolla,ND,48.8844,-99.6208
4,K08D,1.422184,4.632801,0.0,0.000000,1.725647,3.594939,0.079909,Stanley,ND,48.3008,-102.4064


# Visualization

In [20]:
fig_sum = px.histogram(df2, x='Type', y= 'Duration',  histfunc = 'avg', title = 'fig 1. National wide weather events duration')

fig_sum.update_xaxes(categoryorder='mean descending')
fig_sum.update_yaxes(title_text='mean of duration% per year')
fig_sum.show()

## City wide weather events distribution

In [33]:
fig_city = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa", title ='fig 2. Cities involved in this dataset')

fig_city.show()

#### RAIN

In [24]:
fig_rain = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng', color="Rain",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense',scope="usa", 
                      title ='fig 3. City wide rainy days percentage')
fig_rain.show()

## FOG

In [25]:
fig_fog = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',color="Fog", 
                         hover_name=weather['City'] + ', ' + weather['State'],
                         color_continuous_scale='dense', scope="usa", 
                         title ='fig 4. City wide foggy days percentage')
fig_fog.show()

## SNOW

In [26]:
fig_snow = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',color="Snow",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense',scope="usa",
                      title ='fig 5. City wide snow days percentage')
fig_snow.show()

## COLD

In [27]:
fig_cold = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng', color="Cold",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', scope="usa",
                      title ='fig 6. City wide cold days percentage')
fig_cold.show()

## K-means Clustering

In [28]:
X = df_f.drop(['AirportCode','Cold', 'Hail'], axis=1)
X.head()

,Fog,Precipitation,Rain,Snow,Storm
0,2.501237,0.240677,8.976598,0.199153,0.000000
1,0.113680,0.011653,2.005042,1.913194,0.082572
2,2.062215,0.039051,4.820538,3.784484,0.000999
3,2.493341,0.026636,2.018645,5.493721,0.136035
4,4.632801,0.000000,1.725647,3.594939,0.079909


In [29]:
from sklearn.cluster import KMeans

distortions = []
K = range(1,20)
for k in K:
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 500)
    kmean.fit(X)
    distortions.append(kmean.inertia_)

fig_kmean = px.scatter(x=K, y=distortions, trendline=distortions, title='fig 7. The Elbow Method')
fig_kmean.update_xaxes(title_text='k')
fig_kmean.update_yaxes(title_text='distortion')
#fig_kmean.update_layout(height=650, width=1000)
fig_kmean.show()

###### What is the Elbow Method ? 



In [30]:
#results
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

df_f['Cluster'] = (kmeans.labels_).astype(str)
df_cluster = pd.merge(df_f[['AirportCode','Cluster']], weather.drop(['Cold','Hail'], axis=1), 
                      how='inner', on='AirportCode')
df_cluster.head()

,AirportCode,Cluster,Fog,Precipitation,Rain,Snow,Storm,City,State,LocationLat,LocationLng
0,K01M,3,2.501237,0.240677,8.976598,0.199153,0.000000,Belmont,MS,34.4917,-88.2011
1,K04V,2,0.113680,0.011653,2.005042,1.913194,0.082572,Saguache,CO,38.0972,-106.1689
2,K04W,0,2.062215,0.039051,4.820538,3.784484,0.000999,Hinckley,MN,46.0244,-92.8991
3,K06D,0,2.493341,0.026636,2.018645,5.493721,0.136035,Rolla,ND,48.8844,-99.6208
4,K08D,2,4.632801,0.000000,1.725647,3.594939,0.079909,Stanley,ND,48.3008,-102.4064


In [31]:
prop = df_cluster[['Cluster','Fog','Precipitation','Rain','Snow','Storm']].groupby(['Cluster']).mean().reset_index()
prop2 = prop.transpose().reset_index()
prop2 = prop2[(prop2['index'] !='Cluster')].sort_values(by=0)
prop2

,index,0,1,2,3
2,Precipitation,0.0436569,0.0938048,0.0622476,0.0872664
5,Storm,0.0686254,0.0351753,0.101093,0.00994848
1,Fog,2.28446,2.4576,1.20915,2.44403
4,Snow,4.60755,0.520622,0.99112,0.935139
3,Rain,5.47702,5.21039,1.9474,8.62226


In [32]:
fig_prop=make_subplots(rows=1, cols=4, shared_yaxes=True,horizontal_spacing=0)

fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[0], name='Cluster 0'), row=1, col=1)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[1], name='Cluster 1'), row=1, col=2)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[2], name='Cluster 2'), row=1, col=3)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[3], name='Cluster 3'), row=1, col=4)

fig_prop.update_yaxes(title_text="duration%/year", row=1, col=1)
fig_prop.update_layout(title_text="fig 8. Weather distribution in each cluster")
fig_prop.show()

##### Property of each cluster
The property of each cluster is summarized in figure 8. Compare to other clusters, cluster 3 (purple) has highest chance of rain, cluster 2 (green) has highest chance of snow, cluster 1 (red) has lowest chance of rain while highest chance of storm; cluster 0 (blue) is similar to cluster 3 while its’ chance of rain is slightly lower than cluster 3.

In [34]:
fig_cluster = px.scatter_geo(df_cluster, lat='LocationLat', lon='LocationLng',
                      hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa",color_discrete_sequence =['#AB63FA', '#EF553B', '#00CC96','#636EFA'],
                      color = 'Cluster',title ='fig 9. City wide weather cluster distribution')

fig_cluster.update_layout(height=750, width=1000)
fig_cluster.show()

In [35]:
df_cluster2 = df_cluster.groupby(['State','Cluster']).agg({'Cluster':['count']}).reset_index()
df_cluster2.columns=pd.MultiIndex.from_tuples((("State", " "),("Cluster", " "),("Count", " ")))
df_cluster2.columns = df_cluster2.columns.get_level_values(0)
#state with each cluster counts

df_loc = df_cluster[['State','Cluster','LocationLat', 'LocationLng']]
df_loc1 = df_loc.groupby(['State','Cluster']).agg({'LocationLat':'mean'}).reset_index()
df_loc2 = df_loc.groupby(['State','Cluster']).agg({'LocationLng':'mean'}).reset_index()
df_loc3 = pd.merge(df_loc1,df_loc2, how='inner', on=['State','Cluster'])
#state with cluster and location

df_clusterS = pd.merge(df_loc3,df_cluster2, how='inner', on=['State','Cluster'])
df_clusterS.head() #state with each cluster count location

,State,Cluster,LocationLat,LocationLng,Count
0,AL,1,32.040042,-87.053792,12
1,AL,3,33.151254,-86.506113,24
2,AR,1,35.424294,-92.872506,16
3,AR,2,35.563267,-92.754633,3
4,AR,3,34.994525,-92.244744,16


In [36]:
fig_clusterS = px.scatter_geo(df_clusterS, lat='LocationLat', lon='LocationLng', color='Cluster', size='Count',
                     color_discrete_sequence=['#636EFA', '#AB63FA', '#EF553B','#00CC96'], hover_name='State',
                     scope="usa", title = 'fig 10. State wide weather cluster distribution')

fig_clusterS.show()

##### representive cities in each cluster
Four cities were selected in each cluster, Kansas City, MO is the representive city of clsuter 0; Dever, CO is the representive city of cluster 1; Detroit, MI is the representive city of cluster 2; Seattle, MA is the representive city of cluster 3. All the cities weather information are shown in figure 10, their weather properties are similar to their corresponding cluster as shown in figure 10.

In [37]:
df3 = weather[(weather['City']=='Seattle')| (weather['City']=='Detroit')|(weather['City']== 'Kansas City')|(weather['City']== 'Denver')]

df_cities = df3[['Storm', 'Precipitation','Snow', 'Fog','Rain', 'City']].groupby(['City']).mean().reset_index()
df_cities = df_cities.transpose().reset_index()
df_cities.columns = df_cities.iloc[0]
df_cities = df_cities[(df_cities['City'] !='City')]
df_cities

,City,Denver,Detroit,Kansas City,Seattle
1,Storm,0.127997,0.0146499,0.00313927,0.00130803
2,Precipitation,0.0259228,0.075723,0.121401,0.0154585
3,Snow,3.42337,3.6542,0.949534,0.472817
4,Fog,1.90449,2.90539,1.0193,1.73923
5,Rain,2.59237,7.53589,4.45416,12.4189


In [38]:
fig_city=make_subplots(rows=1, cols=4, shared_yaxes=True,horizontal_spacing=0)

fig_city.add_trace(go.Bar(x=df_cities['City'], y=df_cities['Kansas City'], name='Cluster0'), row=1, col=1)
fig_city.add_trace(go.Bar(x=df_cities['City'], y=df_cities['Denver'], name='Cluster1'), row=1, col=2)
fig_city.add_trace(go.Bar(x=df_cities['City'], y=df_cities['Detroit'], name='Cluster2'), row=1, col=3)
fig_city.add_trace(go.Bar(x=df_cities['City'], y=df_cities['Seattle'], name='Cluster3'), row=1, col=4)

fig_city['layout']['xaxis1'].update(title="Kansas City, MO")
fig_city['layout']['xaxis2'].update(title="Denver, CO")
fig_city['layout']['xaxis3'].update(title="Detroit, MI")
fig_city['layout']['xaxis4'].update(title="Seattle, WA")

fig_city.update_yaxes(title_text="duration% / year", row=1, col=1)
fig_city.update_layout(title_text="fig 11. Representative cities in each cluster")

fig_city.show()